# Import Packages

In [ ]:
# Base packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

# Holoviews packages
import holoviews as hv
import hvplot.pandas
from bokeh.io import output_notebook
from bokeh.palettes import varying_alpha_palette
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.transform import linear_cmap
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, NumeralTickFormatter, TableColumn, DataTable, Div, TabPanel, Tabs
output_notebook()
hv.extension('bokeh')
import panel as pn
pn.extension(throttled=True)

# Post processing file
import TEVA_Post_Processing as post

# Import TEVA Output Files

In [ ]:
# Import CC and DNF output files
ccs = pd.read_excel('ccs_2DOC_CAMELS.xlsx', sheet_name='CCEA_Low')
dnfs = pd.read_excel('dnfs_2DOC_CAMELS.xlsx', sheet_name='DNFEA_Low')

# Import observation data
data = pd.read_csv('test_observations.csv')

# Run Post-Processing Functions
These functions, imported from the "TEVA_Post_Processing.py" file, help parse the Excel spreadsheets into a more usable form.

In [ ]:
# List of the CCs composing each DNF
all_ccs = post.flatten(post.parse_dnf(dnfs))

# List of the features composing each CC
cc_features = post.parse_cc(ccs)

# Flatten list of lists
# all_ccs_flat = post.flatten(all_ccs)
# cc_features_flat = flatten(cc_features)

# List of the unique CCs across all DNFs
unique_ccs = (np.unique(all_ccs))

# Fitness contours
x_fit, y_fit, z_fit, fitness = post.fitness_contours(1000, dnfs, ccs)

# CC and DNF lengths
cc_len = np.arange(1, max(ccs['order']) + 1 , 1)
dnf_len = np.arange(1, max(dnfs['order']) + 1 , 1)

# Set up the Dashboard
    > Custom color maps for cc/dnf plot
    

In [ ]:
# Custom color maps
'''
Create custom colormaps, one for CCs, one from DNFs, and one for fitness contours.
Colormaps can range from 0 to 256, but it is best to trim the lightest and darkest portions out.
'''
cc_colors = []
dnf_colors = []

# CCs and DNFs
for i in range(20,220):
    cc_colors.append(colors.rgb2hex(plt.get_cmap('Blues_r')(i)))
    dnf_colors.append(colors.rgb2hex(plt.get_cmap('Oranges_r')(i)))

# Fitness contours
contour_colors = varying_alpha_palette(color='black', start_alpha=150, end_alpha=10)

In [ ]:
# Bokeh Data Sources
'''
Bokeh uses a data structure called a "Column Data Source" (CDS) for plotting.
The easiest way to create them with your data is by passing your data as a disctionary.
'''

# column data source for CCs
cc_plot_data = {'x_values': ccs['cov'],
                'y_values': ccs['ppv'],
                'CC': ccs['Unnamed: 0'],
                'Order': ccs['order'],
                'Features': cc_features}
cc_plot_source = ColumnDataSource(data=cc_plot_data)

# column data source for DNFs
dnf_plot_data = {'x_values': dnfs['cov'],
                 'y_values': dnfs['ppv'],
                 'Order': dnfs['order'],
                 'DNF': dnfs['Unnamed: 0'],
                 'CCs': all_ccs}
dnf_plot_source = ColumnDataSource(data=dnf_plot_data)

# column data source for fitness contours
dnf_cont_data = {'x_values': x_fit,
                 'y_values': y_fit,
                 'z_values': z_fit}
dnf_cont_source = ColumnDataSource(dnf_cont_data)

In [ ]:
# Bokeh Figure Setup
# tooltips to display when you hover over a data point
h = 900
w = 800

dnf_TOOLS = [
    ('DNF #', '@DNF'),
    ('Order', '@Order'),
    ('PPV', '@y_values'),
    ('COV', '@x_values'),
    ('CCs', '@CCs')]

cc_TOOLS = [
    ('CC #', '@CC'),
    ('Order', '@Order'),
    ('PPV', '@y_values'),
    ('COV', '@x_values'),
    ('Features', '@Features')]

#### Figure 1
p1 = figure(width = w, height = h,
           y_range=(0,1.05),
           x_range=(0,1.05),
           x_axis_label='Observation Coverage',
           y_axis_label='Positive Predictive Value',
           hidpi=True,
           tools='crosshair, pan, tap, wheel_zoom, zoom_in, zoom_out, box_zoom, undo, redo, reset, save, lasso_select, help')

# cont_levels = np.linspace(min(fitness), max(fitness), 10)
cont_levels = np.linspace(min(fitness), max(fitness), 10)
contour_renderer = p1.contour(x_fit, y_fit, z_fit,
                             levels=cont_levels,
                             line_color='gray',
                             fill_color=contour_colors,
                             line_dash='dashed')
# Plot CCs, colored by order
for i in range(0, len(cc_len)):
    cc_plots = p1.scatter('x_values', 'y_values', source=cc_plot_source,
              view=CDSView(filter=GroupFilter(column_name='Order', group=len(cc_len) - i)),
              size=12,
              marker='square',
              line_color='white',
              fill_color=linear_cmap('Order', cc_colors, low=min(ccs['order']), high=max(ccs['order'])),
              hover_color='black',
              legend_label='CC Order {}'.format(len(cc_len) - i),
              fill_alpha=1)

# Add hover tool for CCs
p1.add_tools(HoverTool(tooltips=cc_TOOLS,
                      mode='mouse',
                      point_policy='follow_mouse'))

# Plot DNFs, colored by order
all_dnf_plots = []
for i in range(0, len(dnf_len)):
    dnf_plot = p1.scatter('x_values', 'y_values', source=dnf_plot_source,
              view=CDSView(filter=GroupFilter(column_name='Order', group=len(dnf_len) - i)),
              size=13,
              marker='circle',
              line_color='white',
              fill_color=linear_cmap('Order', dnf_colors, low=min(dnfs['order']), high=max(dnfs['order'])),
              hover_color='black',
              legend_label='DNF Order {}'.format(len(dnf_len) - i),
              fill_alpha=1)
    all_dnf_plots.append(dnf_plot)

# Add seperate hover tool for DNFs
p1.add_tools(HoverTool(renderers = all_dnf_plots,
                      tooltips=dnf_TOOLS,
                      mode='mouse',
                      point_policy='follow_mouse'))

# Add color bar for fitness contours
colorbar = contour_renderer.construct_color_bar(height=int(h/2),
                                                location=(0,int(h/4)),
                                                formatter = NumeralTickFormatter(format='0 a'),
                                                bar_line_color='black',
                                                major_tick_line_color='black')

# General formatting
p1.legend.click_policy='hide'
p1.legend.location='bottom_left'
p1.add_layout(colorbar, 'right')
nonselection_fill_alpha=0.2

In [ ]:
# Function for updating plots in the CC explorer
def feature_plotter(selected_cc):
    fig = []
    for i in range(len(cc_features[selected_cc])):
        fig.append(data[cc_features[selected_cc][i]].dropna().hvplot.kde(height=200, width=300, hover=False))
        fig[i].opts(shared_axes=False, toolbar=None)
    return pn.FlexBox(objects=fig)

# Widget for selecting CC to plot
dropdown_options = list(np.sort(unique_ccs.astype(int)))
cc_select = pn.widgets.Select(options=dropdown_options, width=75, name='CC', description='Select a CC to view features.')

# Bind function to widget
dynamic_subplots = pn.bind(feature_plotter, cc_select)

# Create layout
CC_Explorer = pn.Row(cc_select, dynamic_subplots)

In [ ]:
# Assemble dashboard
# app = pn.Row(p1, CC_Explorer)
bokeh_pane = pn.pane.Bokeh(p1)

In [ ]:
pn.Row(bokeh_pane, CC_Explorer)